In [5]:
import sqlite3
from sqlite3 import Error
import pandas as pd


def sql_connection(filename="14_FIFA21.db"):
    try:
        con = sqlite3.connect(filename)
        print("Connection to", filename + '\n\n')
    except Error:
        print(Error)
        con = None
    finally:
        return con

def csv_db(con):
    try:
      csv_data = pd.read_csv('14_FIFA21.csv')
      csv_data.to_sql('player', con)
      return True
    except ValueError:
      print('Tabla player ya creada, pasamos al análisis de los datos. \n\n')
      return True
    except FileNotFoundError:
      print('Fichero csv no encontrado')
      return False

# Creamos la conexión y cargamos los datos
con = sql_connection()
if csv_db(con): 
  cur = con.cursor()
  # Todas las consultas sql han sido programadas en SQLite studio y probadas ahí
  # antes de copiar y pegarlas
  try:
    # Hacemos la consulta con SELECT y WHERE, para ver en orden descendiente los 
    # paises que tienen una media del overall de sus jugadores mayor que 70
    cur.execute("""SELECT nationality,
                    AVG(overall) AS media
                  FROM player
                  GROUP BY nationality
                  HAVING AVG(overall) > 70
                  ORDER BY media DESC;""")
    print('Paises en orden descendiente por media del overall de los juadores que tienen overall mayor que 70:\n')
    for row in cur.fetchall():
      print(f'País: {row[0]:25}| Media: {row[1]:.2f}')
  # Consulta extra por curiosidad, para ver que paises tienen un mayor porcentaje de zurdos
    cur.execute("""SELECT die.nationality,
                        diestros,
                        zurdos,
                        (CAST (zurdos AS FLOAT) / CAST (zurdos + diestros AS FLOAT))*100 AS porcentaje_zurdos
                    FROM (
                            SELECT nationality,
                                    COUNT(CASE WHEN [Preferred Foot] = 'Right' THEN 1 END) AS diestros
                              FROM player
                              GROUP BY nationality
                        )
                        AS die,
                        (
                            SELECT nationality,
                                    COUNT(CASE WHEN [Preferred Foot] = 'Left' THEN 1 END) AS zurdos
                              FROM player
                              GROUP BY nationality
                        )
                        AS zur
                  WHERE die.nationality = zur.nationality
                  ORDER BY porcentaje_zurdos DESC;""") 
    print('\n\nPaises en orden descendiente con una mayor porcentaje de zurdos, consulta por curiosidad:\n')
    for row in cur.fetchall():
        print(f'País: {row[0]:25}| Diestros: {row[1]:4} | Zurdos: {row[2]:4} | Porcentaje de zurdos: {row[3]:.2f}%')

    # Consulta con UPDATE, vamos a reducir el valor de los jugadores que tengan un 
    # potencial menor que su orevall actual con un cociente reductor en función del 
    # cociente entre potential y overall
    cur.execute("""UPDATE player
                      SET value = value * 0.75, [Release Clause] = [Release Clause] * 0.75
                    WHERE potential <= overall AND age > 42;""") 
    print('\n\nActualizados los jugadores que se espere que tengan una perdida de potencial o mantengan el potencial y sean mayores de 42 años.\n')

    # Consulta SELECT que muestra el mayor y el menor overall medio de los jugadores
    # por club
    cur.execute("""SELECT club,
                            overall_medio
                        FROM (
                                SELECT club,
                                        AVG(overall) AS overall_medio
                                  FROM player
                                  GROUP BY club
                            )
                      WHERE overall_medio = (
                                                SELECT MAX(overall_medio) 
                                                  FROM (
                                                            SELECT AVG(overall) AS overall_medio
                                                              FROM player
                                                            WHERE club NOT LIKE '1%'
                                                            GROUP BY club
                                                            HAVING overall_medio > 0
                                                        )
                                            )
                      OR 
                            overall_medio = (
                                                SELECT MIN(overall_medio) 
                                                  FROM (
                                                            SELECT AVG(overall) AS overall_medio
                                                              FROM player
                                                            WHERE club NOT LIKE '1%'
                                                            GROUP BY club
                                                            HAVING overall_medio > 0
                                                        )
                                            );""") 
    print('\n\nClub con mayor overall medio de los jugadores seguido del club con menor overall medio de los jugadores:\n')
    for row in cur.fetchall():
        print(f'Club: {row[0]:20}| Overall medio: {row[1]:.2f}')
    
  except sqlite3.OperationalError:
    print('Error al hacer alguna consulta.')
con.close()



Connection to 14_FIFA21.db


Tabla player ya creada, pasamos al análisis de los datos. 


Paises en orden descendiente por media del overall de los juadores que tienen media mayor que 70:

País: Tanzania                 | Media: 76.00
País: Fiji                     | Media: 73.00
País: Mozambique               | Media: 72.67
País: Syria                    | Media: 72.33
País: Bermuda                  | Media: 72.00
País: Brazil                   | Media: 71.90
País: Algeria                  | Media: 71.39
País: Egypt                    | Media: 71.20
País: New Caledonia            | Media: 71.00
País: Bahrain                  | Media: 71.00
País: Ukraine                  | Media: 70.68
País: Iran                     | Media: 70.59
País: Portugal                 | Media: 70.41
País: Czech Republic           | Media: 70.16


Paises en orden descendiente con una mayor porcentaje de zurdos, consulta por curiosidad:

País: Indonesia                | Diestros:    0 | Zurdos:    1 | Porcentaj